# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 23 2022 8:53AM,240265,10,0085795273,ISDIN Corporation,Released
1,Jun 23 2022 8:53AM,240265,10,0085795294,ISDIN Corporation,Released
2,Jun 23 2022 8:53AM,240265,10,0085795356,ISDIN Corporation,Released
3,Jun 23 2022 8:53AM,240265,10,0085795357,ISDIN Corporation,Released
4,Jun 23 2022 8:53AM,240265,10,0085795398,ISDIN Corporation,Released
5,Jun 23 2022 8:53AM,240265,10,0085795397,ISDIN Corporation,Released
6,Jun 23 2022 8:53AM,240265,10,0085795400,ISDIN Corporation,Released
7,Jun 23 2022 8:53AM,240265,10,0085795505,ISDIN Corporation,Released
8,Jun 23 2022 8:53AM,240265,10,0085795507,ISDIN Corporation,Released
9,Jun 23 2022 8:53AM,240265,10,0085795511,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,240260,Released,35
30,240261,Released,34
31,240263,Released,27
32,240264,Released,1
33,240265,Released,42


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240260,NaN,NaN,35.0
240261,NaN,NaN,34.0
240263,NaN,NaN,27.0
240264,NaN,NaN,1.0
240265,NaN,NaN,42.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0.0,0.0,1.0
240161,0.0,0.0,1.0
240194,0.0,0.0,1.0
240197,0.0,1.0,0.0
240200,1.0,10.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0,0,1
240161,0,0,1
240194,0,0,1
240197,0,1,0
240200,1,10,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,0,0,1
1,240161,0,0,1
2,240194,0,0,1
3,240197,0,1,0
4,240200,1,10,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,,,1
1,240161,,,1
2,240194,,,1
3,240197,,1,
4,240200,1,10,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 23 2022 8:53AM,240265,10,ISDIN Corporation
42,Jun 23 2022 8:33AM,240264,10,Bio-PRF
43,Jun 23 2022 8:33AM,240263,10,ISDIN Corporation
70,Jun 23 2022 8:32AM,240261,10,ISDIN Corporation
104,Jun 23 2022 8:32AM,240260,10,ISDIN Corporation
139,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc."
140,Jun 23 2022 8:23AM,240259,10,ISDIN Corporation
149,Jun 23 2022 8:00AM,240258,19,"AdvaGen Pharma, Ltd"
150,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd"
167,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 23 2022 8:53AM,240265,10,ISDIN Corporation,,,42
1,Jun 23 2022 8:33AM,240264,10,Bio-PRF,,,1
2,Jun 23 2022 8:33AM,240263,10,ISDIN Corporation,,,27
3,Jun 23 2022 8:32AM,240261,10,ISDIN Corporation,,,34
4,Jun 23 2022 8:32AM,240260,10,ISDIN Corporation,,,35
5,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc.",,,1
6,Jun 23 2022 8:23AM,240259,10,ISDIN Corporation,,,9
7,Jun 23 2022 8:00AM,240258,19,"AdvaGen Pharma, Ltd",,,1
8,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",,12,5
9,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 8:53AM,240265,10,ISDIN Corporation,42,,
1,Jun 23 2022 8:33AM,240264,10,Bio-PRF,1,,
2,Jun 23 2022 8:33AM,240263,10,ISDIN Corporation,27,,
3,Jun 23 2022 8:32AM,240261,10,ISDIN Corporation,34,,
4,Jun 23 2022 8:32AM,240260,10,ISDIN Corporation,35,,
5,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc.",1,,
6,Jun 23 2022 8:23AM,240259,10,ISDIN Corporation,9,,
7,Jun 23 2022 8:00AM,240258,19,"AdvaGen Pharma, Ltd",1,,
8,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",5,12,
9,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 8:53AM,240265,10,ISDIN Corporation,42,,
1,Jun 23 2022 8:33AM,240264,10,Bio-PRF,1,,
2,Jun 23 2022 8:33AM,240263,10,ISDIN Corporation,27,,
3,Jun 23 2022 8:32AM,240261,10,ISDIN Corporation,34,,
4,Jun 23 2022 8:32AM,240260,10,ISDIN Corporation,35,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 8:53AM,240265,10,ISDIN Corporation,42.0,NaN,NaN
1,Jun 23 2022 8:33AM,240264,10,Bio-PRF,1.0,NaN,NaN
2,Jun 23 2022 8:33AM,240263,10,ISDIN Corporation,27.0,NaN,NaN
3,Jun 23 2022 8:32AM,240261,10,ISDIN Corporation,34.0,NaN,NaN
4,Jun 23 2022 8:32AM,240260,10,ISDIN Corporation,35.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 8:53AM,240265,10,ISDIN Corporation,42.0,0.0,0.0
1,Jun 23 2022 8:33AM,240264,10,Bio-PRF,1.0,0.0,0.0
2,Jun 23 2022 8:33AM,240263,10,ISDIN Corporation,27.0,0.0,0.0
3,Jun 23 2022 8:32AM,240261,10,ISDIN Corporation,34.0,0.0,0.0
4,Jun 23 2022 8:32AM,240260,10,ISDIN Corporation,35.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2402396,152.0,0.0,0.0
15,240205,0.0,1.0,0.0
16,720672,3.0,0.0,0.0
19,1441355,14.0,12.0,7.0
20,1201101,5.0,39.0,2.0
21,480408,1.0,1.0,0.0
22,240218,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2402396,152.0,0.0,0.0
1,15,240205,0.0,1.0,0.0
2,16,720672,3.0,0.0,0.0
3,19,1441355,14.0,12.0,7.0
4,20,1201101,5.0,39.0,2.0
5,21,480408,1.0,1.0,0.0
6,22,240218,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,152.0,0.0,0.0
1,15,0.0,1.0,0.0
2,16,3.0,0.0,0.0
3,19,14.0,12.0,7.0
4,20,5.0,39.0,2.0
5,21,1.0,1.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,152.0
1,15,Released,0.0
2,16,Released,3.0
3,19,Released,14.0
4,20,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,7.0,2.0,0.0,0.0
Executing,0.0,1.0,0.0,12.0,39.0,1.0,1.0
Released,152.0,0.0,3.0,14.0,5.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,7.0,2.0,0.0,0.0
1,Executing,0.0,1.0,0.0,12.0,39.0,1.0,1.0
2,Released,152.0,0.0,3.0,14.0,5.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,7.0,2.0,0.0,0.0
1,Executing,0.0,1.0,0.0,12.0,39.0,1.0,1.0
2,Released,152.0,0.0,3.0,14.0,5.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()